In [3]:
from torchvision.datasets import ImageFolder
import os

root_dir = "/kaggle/input/pacs-dataset/kfold"
all_domains = ['sketch', 'cartoon', 'photo', 'art_painting']

for domain in all_domains:
    folder = os.path.join(root_dir, domain)
    dataset = ImageFolder(folder)
    labels = [label for _, label in dataset.samples]
    print(f"{domain}: Classes = {dataset.classes}")
    print(f"{domain}: Max Label = {max(labels)}, Unique Labels = {sorted(set(labels))}\n")


sketch: Classes = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
sketch: Max Label = 6, Unique Labels = [0, 1, 2, 3, 4, 5, 6]

cartoon: Classes = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
cartoon: Max Label = 6, Unique Labels = [0, 1, 2, 3, 4, 5, 6]

photo: Classes = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
photo: Max Label = 6, Unique Labels = [0, 1, 2, 3, 4, 5, 6]

art_painting: Classes = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
art_painting: Max Label = 6, Unique Labels = [0, 1, 2, 3, 4, 5, 6]



In [1]:
# ✅ Full Updated Code with GRL Fix for Domain Labels

import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from PIL import Image
from sklearn.model_selection import train_test_split

# ============================
# ✅ Reproducibility
# ============================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# ============================
# ✅ Config
# ============================
root_dir = "/kaggle/input/pacs-dataset/kfold"
train_domains = ['sketch', 'cartoon', 'photo']
test_domain = 'art_painting'
domain_to_idx_train = {d: i for i, d in enumerate(train_domains)}
domain_to_idx_full = {'art_painting':0, 'cartoon':1, 'photo':2, 'sketch':3}

batch_size = 32
num_classes = 7
lr = 5e-5
max_epochs = 50
patience = 10
alpha_mixup = 0.4
gamma_margin = 0.5
lambda_domain = 1.0
lambda_margin = 0.2

# ============================
# ✅ Transforms
# ============================
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

# ============================
# ✅ Dataset
# ============================
class PACSDomainDataset(Dataset):
    def __init__(self, samples, labels, domain_labels, transform):
        self.samples = samples
        self.labels = labels
        self.domains = domain_labels
        self.transform = transform
    def __len__(self): return len(self.samples)
    def __getitem__(self, i):
        img = Image.open(self.samples[i]).convert('RGB')
        if self.transform: img = self.transform(img)
        return img, self.labels[i], self.domains[i]

train_samples_all, train_labels_all, train_domains_all = [], [], []
val_samples_all, val_labels_all, val_domains_all = [], [], []

for d in train_domains:
    folder = os.path.join(root_dir, d)
    domain_idx = domain_to_idx_train[d]
    data = ImageFolder(folder)
    paths = [s[0] for s in data.samples]
    labels = [s[1] for s in data.samples]
    train_idx, val_idx = train_test_split(range(len(paths)), test_size=0.2, stratify=labels, random_state=seed)
    for idx in train_idx:
        train_samples_all.append(paths[idx])
        train_labels_all.append(labels[idx])
        train_domains_all.append(domain_idx)
    for idx in val_idx:
        val_samples_all.append(paths[idx])
        val_labels_all.append(labels[idx])
        val_domains_all.append(domain_idx)

train_dataset = PACSDomainDataset(train_samples_all, train_labels_all, train_domains_all, transform_train)
val_dataset = PACSDomainDataset(val_samples_all, val_labels_all, val_domains_all, transform_test)

test_paths = [s[0] for s in ImageFolder(os.path.join(root_dir, test_domain)).samples]
test_labels = [s[1] for s in ImageFolder(os.path.join(root_dir, test_domain)).samples]
test_domains = [domain_to_idx_full[test_domain]] * len(test_paths)
test_dataset = PACSDomainDataset(test_paths, test_labels, test_domains, transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"Train size: {len(train_dataset)}, Val size: {len(val_dataset)}, Test size: {len(test_dataset)}")

# ============================
# ✅ GRL
# ============================
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambd):
        ctx.lambd = lambd
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.lambd, None

class GRL(nn.Module):
    def __init__(self, lambd=1.0):
        super().__init__()
        self.lambd = lambd
    def forward(self, x):
        return GradReverse.apply(x, self.lambd)

# ============================
# ✅ Models
# ============================
class FIXEDFeatureExtractor(nn.Module):
    def __init__(self, backbone, feat_dim):
        super().__init__()
        self.backbone = backbone
        self.domain_invariant_proj = nn.Sequential(
            nn.Linear(feat_dim, feat_dim),
            nn.BatchNorm1d(feat_dim),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
    def forward(self, x):
        raw_feats = self.backbone(x)
        inv_feats = self.domain_invariant_proj(raw_feats)
        return raw_feats, inv_feats

class FIXEDClassifier(nn.Module):
    def __init__(self, feat_dim, num_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(feat_dim, feat_dim//2),
            nn.BatchNorm1d(feat_dim//2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(feat_dim//2, num_classes)
        )
    def forward(self, x): return self.fc(x)

class DomainClassifier(nn.Module):
    def __init__(self, feat_dim, num_domains):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(feat_dim, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.1),
            nn.Linear(256, num_domains)
        )
    def forward(self, x): return self.net(x)

# ============================
# ✅ Init models (ResNet18)
# ============================
backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
feat_dim = backbone.fc.in_features
backbone.fc = nn.Identity()

feature_extractor = FIXEDFeatureExtractor(backbone, feat_dim).to(device)
classifier = FIXEDClassifier(feat_dim, num_classes).to(device)
domain_classifier = DomainClassifier(feat_dim, len(train_domains)).to(device)
grl = GRL()

optimizer = torch.optim.Adam([
    {'params': feature_extractor.parameters(), 'lr': lr},
    {'params': classifier.parameters(), 'lr': lr},
    {'params': domain_classifier.parameters(), 'lr': lr * 10}
], weight_decay=1e-4)

criterion_cls = nn.CrossEntropyLoss()
criterion_domain = nn.CrossEntropyLoss()

# ============================
# ✅ Loss helpers
# ============================
def large_margin_loss(logits, labels, gamma=0.5):
    correct = logits.gather(1, labels.view(-1,1)).squeeze()
    mask = torch.ones_like(logits).scatter(1, labels.view(-1,1), 0.0)
    max_incorrect = (logits*mask + (1-mask)*(-1e8)).max(1)[0]
    return F.relu(gamma + max_incorrect - correct).mean()

def mixup_data(features, labels, alpha=0.4):
    lam = np.random.beta(alpha, alpha)
    index = torch.randperm(features.size(0)).to(device)
    mixed = lam * features + (1-lam) * features[index]
    return mixed, labels, labels[index], lam

def mixup_criterion(crit, preds, y_a, y_b, lam):
    return lam * crit(preds, y_a) + (1-lam) * crit(preds, y_b)

# ============================
# ✅ Training with patience
# ============================
best_val_acc = 0.0
epochs_no_improve = 0

for epoch in range(1, max_epochs+1):
    feature_extractor.train()
    classifier.train()
    domain_classifier.train()
    total_loss = 0.0

    for imgs, labels, d_labels in train_loader:
        imgs, labels, d_labels = imgs.to(device), labels.to(device), d_labels.to(device)
        optimizer.zero_grad()
        raw_feats, inv_feats = feature_extractor(imgs)
        mixed_feats, y_a, y_b, lam = mixup_data(inv_feats, labels, alpha_mixup)
        cls_logits = classifier(mixed_feats)
        loss_cls = mixup_criterion(criterion_cls, cls_logits, y_a, y_b, lam)
        loss_margin = large_margin_loss(cls_logits, y_a, gamma_margin)
        dom_logits = domain_classifier(grl(raw_feats))
        loss_domain = criterion_domain(dom_logits, d_labels)
        loss = loss_cls + lambda_domain * loss_domain + lambda_margin * loss_margin
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    feature_extractor.eval()
    classifier.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels, _ in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            _, feats = feature_extractor(imgs)
            preds = classifier(feats).argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_acc = 100.0 * correct / total

    print(f"Epoch {epoch}/{max_epochs} - Train Loss: {total_loss/len(train_loader):.4f} | Val Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        torch.save({
            'feature_extractor': feature_extractor.state_dict(),
            'classifier': classifier.state_dict()
        }, "best_model.pth")
        print(f"  ✅ New best val acc: {best_val_acc:.2f}% (model saved)")
    else:
        epochs_no_improve += 1
        print(f"  No improvement. Patience: {epochs_no_improve}/{patience}")
        if epochs_no_improve >= patience:
            print(f"⏹ Early stopping at epoch {epoch}. Best Val Acc: {best_val_acc:.2f}%")
            break

# ============================
# ✅ Final test evaluation
# ============================
ckpt = torch.load("best_model.pth", map_location=device)
feature_extractor.load_state_dict(ckpt['feature_extractor'])
classifier.load_state_dict(ckpt['classifier'])
feature_extractor.eval()
classifier.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels, _ in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        _, feats = feature_extractor(imgs)
        preds = classifier(feats).argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = 100.0 * correct / total
print(f"\n✅ Final Test Accuracy on {test_domain}: {test_acc:.2f}%")


Using device: cuda
Train size: 6354, Val size: 1589, Test size: 2048


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 209MB/s]


Epoch 1/50 - Train Loss: 2.4465 | Val Acc: 76.15%
  ✅ New best val acc: 76.15% (model saved)
Epoch 2/50 - Train Loss: 2.3817 | Val Acc: 80.11%
  ✅ New best val acc: 80.11% (model saved)
Epoch 3/50 - Train Loss: 2.2223 | Val Acc: 88.99%
  ✅ New best val acc: 88.99% (model saved)
Epoch 4/50 - Train Loss: 2.2552 | Val Acc: 89.24%
  ✅ New best val acc: 89.24% (model saved)
Epoch 5/50 - Train Loss: 2.1539 | Val Acc: 90.43%
  ✅ New best val acc: 90.43% (model saved)
Epoch 6/50 - Train Loss: 2.0911 | Val Acc: 92.32%
  ✅ New best val acc: 92.32% (model saved)
Epoch 7/50 - Train Loss: 2.1173 | Val Acc: 92.07%
  No improvement. Patience: 1/10
Epoch 8/50 - Train Loss: 2.0830 | Val Acc: 93.14%
  ✅ New best val acc: 93.14% (model saved)
Epoch 9/50 - Train Loss: 2.0859 | Val Acc: 91.94%
  No improvement. Patience: 1/10
Epoch 10/50 - Train Loss: 2.0795 | Val Acc: 93.08%
  No improvement. Patience: 2/10
Epoch 11/50 - Train Loss: 2.0318 | Val Acc: 93.83%
  ✅ New best val acc: 93.83% (model saved)
Epoch